In [1]:
import pandas as pd
import numpy as np
import os


import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
import statsmodels.api as sm

# importing required modules
from zipfile import ZipFile
import zipfile
import time

In [2]:
# file_name = 'stataversion.dta'
# returns_data = pd.read_stata(file_name) #reads in the data file 
# returns_data.head()

In [3]:
# file_name2 = 'hpbx8j61yn2jkyar.dta'
# fundamental_data = pd.read_stata(file_name2) #reads in the data file 
# fundamental_data.head()

In [4]:
# # specifying the zip file name
# file_name = "fundamentals.zip"
  
# # opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()
  
#     print('Extracting all the files now...')
#     zip.extractall()
#     #data = zip.read('hpbx8j61yn2jkyar.dta')
#     extracted_file = zipfile.open('hpbx8j61yn2jkyar.dta') #pull the dta file #
    


In [2]:
reader=pd.read_stata("hpbx8j61yn2jkyar.dta",chunksize=100000) #read dta file in chunks

df = pd.DataFrame()

for itm in reader:
    df=df.append(itm)

df.to_csv("fundamentals.csv") #convert it to a csv 

In [4]:
reader=pd.read_stata("stataversion.dta",chunksize=100000)

df = pd.DataFrame()

for itm in reader:
    df=df.append(itm)

df.to_csv("returns.csv")

In [8]:
#Compustat data 
fundamentals = pd.concat((chunk for chunk in pd.read_csv('fundamentals.csv',chunksize=100000, low_memory=False)))
fundamentals.shape #~500k rows, 659 columns 

(501147, 659)

In [20]:
#CRSP data
returns = pd.concat((chunk for chunk in pd.read_csv('returns.csv',chunksize=100000, low_memory=False)))
returns.shape #~5.7 million rows, 16 columns

(5711711, 16)

In [54]:
#cleaning up returns dataset 

returns['date'] = pd.to_datetime(returns['date'], format='%Y%m%d', errors='coerce') #change dates format 
returns.sort_values(by='date') # This now sorts in date order
returns #returns in number format, not percentages 
returns[['prc','vol','ret','shrout','sprtrn']] = returns[['prc','vol','ret','shrout','sprtrn']].apply(pd.to_numeric, errors='coerce') #had to change the num to to numeric values 
#returns['ticker'].nunique() #have stock return data on 9,670 firms 
returns.tail()

,Unnamed: 0,permno,date,siccd,ncusip,ticker,comnam,shrcls,naics,prc,vol,ret,shrout,cfacpr,cfacshr,sprtrn,Gross_Ret
5711706,5711706,93436,2020-12-24,9999,88160R10,TSLA,TESLA INC,NaN,336111.0,661.77002,22813214.0,0.024444,947901.0,1.0,1.0,0.003537,1.024444
5711707,5711707,93436,2020-12-28,9999,88160R10,TSLA,TESLA INC,NaN,336111.0,663.69000,32232732.0,0.002901,947901.0,1.0,1.0,0.008723,1.002901
5711708,5711708,93436,2020-12-29,9999,88160R10,TSLA,TESLA INC,NaN,336111.0,665.98999,22867542.0,0.003465,947901.0,1.0,1.0,-0.002227,1.003465
5711709,5711709,93436,2020-12-30,9999,88160R10,TSLA,TESLA INC,NaN,336111.0,694.78003,42766263.0,0.043229,947901.0,1.0,1.0,0.001342,1.043229
5711710,5711710,93436,2020-12-31,9999,88160R10,TSLA,TESLA INC,NaN,336111.0,705.66998,49544141.0,0.015674,947901.0,1.0,1.0,0.006439,1.015674


In [45]:
fundamentals.sort_values(by='fyearq').reset_index() # This now sorts in date order
fundamentals.tail(15)

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
501132,501132,345920,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501133,501133,345920,2020-09-30,2020.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501134,501134,345920,2020-12-31,2020.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501135,501135,345980,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501136,501136,345980,2018-06-30,2018.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501137,501137,345980,2018-09-30,2018.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501138,501138,345980,2018-12-31,2018.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501139,501139,345980,2019-03-31,2019.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501140,501140,345980,2019-06-30,2019.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501141,501141,345980,2019-09-30,2019.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16


In [22]:
returns.columns
#permno: unique stock (share class) identifier  
#siccd: industry classification code
#comnam: company name 
#shrcls: share class
#naics: north american industry classification code
#prc: price
#vol: volume
#ret: daily return 
#sprtrn: sp500 daily return 

Index(['Unnamed: 0', 'permno', 'date', 'siccd', 'ncusip', 'ticker', 'comnam',
       'shrcls', 'naics', 'prc', 'vol', 'ret', 'shrout', 'cfacpr', 'cfacshr',
       'sprtrn'],
      dtype='object')

In [27]:
#null values for each variable 
print(returns.isnull().sum()) #v_Lot_Frontage has 321 null values 


Unnamed: 0          0
permno              0
date                0
siccd              88
ncusip             88
ticker          12168
comnam             88
shrcls        5283496
naics          101883
prc             15009
vol             15008
ret             17136
shrout           1281
cfacpr           1281
cfacshr          1281
sprtrn              0
dtype: int64


count    501147.000000
mean      93469.379464
std       75628.137085
min        1004.000000
25%       21520.000000
50%       65230.000000
75%      174222.000000
max      345980.000000
Name: gvkey, dtype: float64

In [55]:
#group returns by quarter
#group return dates by 3/31, 6/30, 9/30, 12/31 


returns['Gross_Ret'] = 1 + returns['ret']

q2_2018 = ((returns[(returns['date'] >= '2018-03-31') & (returns['date'] <= '2018-06-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2018 = ((returns[(returns['date'] >= '2018-06-30') & (returns['date'] <= '2018-09-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2018 = ((returns[(returns['date'] >= '2018-09-30') & (returns['date'] <= '2018-12-31')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q1_2019 = ((returns[(returns['date'] >= '2018-12-31') & (returns['date'] <= '2019-03-31')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q2_2019 = ((returns[(returns['date'] >= '2019-03-31') & (returns['date'] <= '2019-06-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2019 = ((returns[(returns['date'] >= '2019-06-30') & (returns['date'] <= '2019-09-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2019 = ((returns[(returns['date'] >= '2019-09-30') & (returns['date'] <= '2019-12-31')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q1_2020 = ((returns[(returns['date'] >= '2019-12-31') & (returns['date'] <= '2020-03-31')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q2_2020 = ((returns[(returns['date'] >= '2020-03-31') & (returns['date'] <= '2020-06-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2020 = ((returns[(returns['date'] >= '2020-06-30') & (returns['date'] <= '2020-09-30')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2020 = ((returns[(returns['date'] >= '2020-09-30') & (returns['date'] <= '2020-12-31')]).groupby(['ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 